Вариант 18

$\begin{equation}
    \begin{cases}
      4x_{1} - \cos(x_{2}) = 0\\
      4x_{2} - e^{x_{1}} = 0
    \end{cases}\,.
\end{equation}$

In [108]:
import numpy as np
import matplotlib.pyplot as plt

eq1 = lambda x1, x2: 4*x1 - np.cos(x2)
eq2 = lambda x1, x2: 4*x2 - np.exp(x1)

Построим на плоскости в интересующей нас области кривые $f_{1}(x_{1}, x_{2}) = 0$ и $f_{2}(x_{1}, x_{2}) = 0$, получим:

In [2]:
eq1_zero = lambda x2: np.cos(x2)/4
eq2_zero = lambda x1: np.exp(x1)/4

eq1_arr_zero = []
eq2_arr_zero = []
x_vals = np.arange(-20, 20, 0.01)
x1_vals_zero = [eq1_zero(x_val) for x_val in x_vals]
x2_vals_zero = [eq2_zero(x_val) for x_val in x_vals]

fig, ax = plt.subplots(figsize=(8,6))
ax.plot(x1_vals_zero, x_vals, color = 'red')
ax.plot(x_vals, x2_vals_zero, color = 'blue')
plt.ylim(-3, 3)  
plt.xlim(-0.3, 0.3)
plt.axhline(0, color='black')
plt.axvline(0, color='black')

NameError: name 'np' is not defined

Из графика понятно, что решение системы уравнений лежит где-то в прямоугольнике $0.2 < x_{1} < 0.3$,   $0.1 < x_{2}< 0.4$

Возьмём для начального приближения точку $x = (0.25, 0.25)$

## Метод Ньютона

Используем функции из предыдущих лабораторных для решения СЛАУ.

In [110]:
def lu_pivot(a):
    N = a.shape[0]
    u = a.copy()
    L = np.eye(N)
    swap_array = [] # в этот массив записываются перестановки строк матрицы
    maxi = 0 
    for j in range(N-1): 
        piv = u[0, :] #строка с максимальным по модулю элементом (pivot)
        for i in range(N): 
            if abs(u[i, j]) > abs(piv[j]):
                piv = u[i, :]
                maxi = i 
        if maxi > j:
            u[[j, maxi], :] = u[[maxi, j], :] #поднимаем строку с максимальным элементом наверх
            swap_array.append([j, maxi]) 
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j] 
        lam[j+1:, j] = -gamma 
        u = lam @ u #домножая на матрицу Лямбда зануляем соответствующий столбец 
        lam[j+1:, j] = gamma #это уже будет матрица обратная к Лямбда, из произведения этих матриц мы строим L
        L = L @ lam 
    return L, u, swap_array

def solve_slau(A, b):
    N = A.shape[0]
    L, u, swap = lu_pivot(A)
    x = np.zeros(N)
    z = np.zeros(N)
    z[0] = b[0] # решаем систему Lz  = b, где z = Ux (прямой ход метода Гаусса)
    for i in range(1, N): 
        z[i] = b[i]
        for j in range(0, i):
            z[i] -= L[i, j] * z[j]
            
    x[N-1] = z[N-1] / u[N-1, N-1] #решаем систему Ux = z (обратный ход)
    for i in range(N-2, -1, -1): #идём с конца
        x[i] = z[i] / u[i, i]
        for j in range(i+1, N):
            x[i] -= (u[i, j] * x[j]) / u[i, i]
    return x, swap

def norm_2_vec(x): # счиатет 2-норму для вектора
    sum_sq = 0
    for y in x:
        sum_sq += y ** 2
    return np.sqrt(sum_sq)

In [111]:
def newton_sys(eq1, eq2, x0, eps):
    x_next = x0
    it = 0
    while True:
        x = x_next
        xk_1 = x[0]
        xk_2 = x[1]
        J_xk = np.array([[4, np.sin(xk_2)], [-np.exp(xk_1), 4]])
        f_xk = np.array([eq1(xk_1, xk_2), eq2(xk_1, xk_2)])
        d_xk, swap = solve_slau(J, -f_xk)
        x_next = x + d_xk
        it+=1
        if norm_2_vec(x_next - x) <= eps:
            break
    return x_next, it

Протестируем функцию:

In [112]:
eps = 1e-10
x0 = [0.25, 0.25]


root, it = newton_sys(eq1, eq2, x0, eps)
print("Корень " + str(root) + " был найден методом Ньютона за " + str(it) + " итераций")

Корень [0.23754123 0.31703182] был найден методом Ньютона за 13 итераций


Подставим значения в исходную систему для проверки:

In [114]:
np.testing.assert_almost_equal(eq1(root[0], root[1]), 0)
np.testing.assert_almost_equal(eq2(root[0], root[1]), 0)

## Метод простых итераций

Возьмём в качестве функций $\phi_{1}$ и $\phi_{2} $ следующие функции: 

$$ \phi_{1} = \frac{\cos(x_{2})}{4} \quad \quad \phi_{2} = \frac{e^{x_{1}}}{4} $$

Проверим выполнение достаточного условия сходимости метода простых итераций в области G - прямоугольнике $0.2 < x_{1} < 0.3$,   $0.1 < x_{2}< 0.4$

$$\frac{\partial \phi_{1}(x_{1},x_{2})}{x_{1}} = 0 \quad\quad \frac{\partial \phi_{1}(x_{1},x_{2})}{x_{2}} = -\frac{\sin(x_{2})}{4} $$

$$\frac{\partial \phi_{2}(x_{1},x_{2})}{x_{1}} = \frac{e^{x_{1}}}{4} \quad\quad \frac{\partial \phi_{2}(x_{1},x_{2})}{x_{2}} = 0$$

В области G имеем:

$$\left\| \frac{\partial \phi_{1}(x_{1},x_{2})}{x_{1}} \right\| + \left\| \frac{\partial \phi_{1}(x_{1},x_{2})}{x_{2}}  \right\| = \left\| -\frac{\sin(x_{2})}{4} \right\| \leqslant 0.25 $$

$$\left\| \frac{\partial \phi_{2}(x_{1},x_{2})}{x_{1}} \right\| + \left\| \frac{\partial \phi_{2}(x_{1},x_{2})}{x_{2}}  \right\| = \left\| \frac{e^{x_{1}}}{4} \right\| \leqslant 0.372956 \approx 0.373 $$ 

Следовательно, если последовательные приближения точки x не покинут точки области G, метод сойдётся, и условие завершения вычислений можно записать в следующем виде:

$$ \frac{q}{1-q} \left| x^{(k+1)} - x^{(k)} \right| = \frac{0.373}{1-0.373} \left| x^{(k+1)} - x^{(k)} \right| \approx 0.595 \left| x^{(k+1)} - x^{(k)} \right| \leqslant \varepsilon $$

In [115]:
phi1 = lambda x2: np.cos(x2)/4
phi2 = lambda x1: np.exp(x1)/4

In [116]:
def simple_iter_sys(phi1, phi2, x0, eps):
    x_next = x0
    it = 0
    while True:
        x = x_next
        x_next = np.array([phi1(x[1]), phi2(x[0])])
        it+=1
        if 0.595*norm_2_vec(x_next - x) <= eps:
            break
    return x_next, it

In [117]:
eps = 1e-10
x0 = [0.5, 0.5]

root, it = simple_iter_sys(phi1, phi2, x0, eps)
print("Корень " + str(root) + " был найден методом простых итераций за " + str(it) + " итераций")

Корень [0.23754123 0.31703182] был найден методом простых итераций за 13 итераций


Проверка

In [118]:
np.testing.assert_almost_equal(eq1(root[0], root[1]), 0)
np.testing.assert_almost_equal(eq2(root[0], root[1]), 0)